# HSE 2022: Mathematical Methods for Data Analysis

## Homework 2

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLSResults
from statsmodels.regression.linear_model import RegressionResultsWrapper
from math import sqrt
import random
import sys

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

sns.set(style="darkgrid")

### Data

For this homework we use Dataset from seaborn on diamonds prices.

In [91]:
data = sns.load_dataset('diamonds')

y = data.price
X = data.drop(['price'], axis=1)
columns = data.drop(['price'], axis=1).columns

## Linear regression

#### 0. [0.25 points] Encode categorical variables.

I used this description https://www.kaggle.com/datasets/shivam2503/diamonds for understanding significance of every variavle.

cut: quality of the cut (Fair, Good, Very Good, Premium, Ideal)

color: from J (worst) to D (best)

clarity: (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))

In [92]:
data.head(5000)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
4995,0.70,Ideal,F,VVS2,61.9,54.8,3741,5.68,5.72,3.53
4996,0.58,Ideal,D,VVS1,62.2,56.0,3741,5.34,5.36,3.33
4997,0.90,Good,I,VVS1,63.9,63.0,3741,6.04,6.07,3.87
4998,1.08,Good,J,SI2,63.2,59.0,3742,6.40,6.57,4.10


In [93]:
data.replace({'cut': {'Fair': 1, 'Good': 2, 'Very Good': 3, 'Premium': 4, 'Ideal': 5}},inplace=True)
data.replace({'color': {'J': 1, 'I': 2, 'H': 3, 'G': 4, 'F': 5, 'E': 6, 'D': 7}}, inplace=True)
data.replace({'clarity': {'I1': 1, 'SI2': 2, 'SI1': 3, 'VS2': 4, 'VS1': 5, 'VVS2': 6, 'VVS1': 7, 'IF': 8}}, inplace=True)
X = data.drop(['price'], axis=1)
data.head(5)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,5,6,2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,4,6,3,59.8,61.0,326,3.89,3.84,2.31
2,0.23,2,6,5,56.9,65.0,327,4.05,4.07,2.31
3,0.29,4,2,4,62.4,58.0,334,4.20,4.23,2.63
4,0.31,2,1,2,63.3,58.0,335,4.34,4.35,2.75


#### 1. [0.25 points] Split the data into train and test sets with ratio 80:20 with random_state=17.

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.80, test_size=0.20, random_state=17)

#### 2. [1 point] Train models on train data using StatsModels library and apply it to the test set; use $RMSE$ and $R^2$ as the quality measure.

* [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html);
* [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with $\alpha = 0.01$;
* [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with $\alpha = 0.01$
* [`ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) with $\alpha = 0.01$, $l_{1}$_$ratio = 0.6$

Don't forget to scale the data before training the models with StandardScaler!

In [95]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Experimentally, it was found that the quality of the model is better with a constant.
X_train_with_const = sm.add_constant(X_train_scaled)
X_test_with_const = sm.add_constant(X_test_scaled)

# Simple Linear Regression.
linear_regression = sm.OLS(y_train, X_train_with_const)
simple_linear_results = linear_regression.fit()
y_pred = simple_linear_results.predict(X_test_with_const)
print("Simple Linear Regression:")
print("Test RMSE = %.4f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Test R^2 = %.4f\n" % r2_score(y_test, y_pred))

# Model with l2 regularization.
ridge_regression = sm.OLS(y_train, X_train_with_const)
ridge_results = ridge_regression.fit_regularized(alpha=0.01, L1_wt=0)
y_pred = ridge_results.predict(X_test_with_const)
print("Model with l2 regularization:")
print("Test RMSE = %.4f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Test R^2 = %.4f\n" % r2_score(y_test, y_pred))

# Model with l1 regularization.
lasso_regression = sm.OLS(y_train, X_train_with_const)
lasso_results = lasso_regression.fit_regularized(alpha=0.01, L1_wt=1, refit=True)

y_pred = lasso_results.predict(X_test_with_const)
print("Model with l1 regularization:")
print("Test RMSE = %.4f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Test R^2 = %.4f\n" % r2_score(y_test, y_pred))

# Model with ElasticNet reqularization.
elastic_regression = sm.OLS(y_train, X_train_with_const)
elastic_results = elastic_regression.fit_regularized(method='elastic_net', alpha=0.01, L1_wt=0.6, refit=True)
print("Model with ElasticNet reqularization:")
print("Test RMSE = %.4f" % np.sqrt(mean_squared_error(y_test, y_pred)))
print("Test R^2 = %.4f\n" % r2_score(y_test, y_pred))

Simple Linear Regression:
Test RMSE = 1240.1860
Test R^2 = 0.9050

Model with l2 regularization:
Test RMSE = 1250.8344
Test R^2 = 0.9034

Model with l1 regularization:
Test RMSE = 1240.1860
Test R^2 = 0.9050

Model with ElasticNet reqularization:
Test RMSE = 1240.1860
Test R^2 = 0.9050



#### 3. [1 point] Explore the values of the parameters of the resulting models and compare the number of zero weights in them. Comment on the significance of the coefficients, overal model significance and other related factors from the results table

Изучите значения параметров результирующих моделей и сравните количество нулевых весов в них. Прокомментируйте значимость коэффициентов, общую значимость модели и другие связанные факторы из таблицы результатов

In [96]:
print("Simple Linear Regression:\n", simple_linear_results.summary2())

Simple Linear Regression:
                   Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.907      
Dependent Variable: price            AIC:                735145.5726
Date:               2022-10-16 20:24 BIC:                735232.2974
No. Observations:   43152            Log-Likelihood:     -3.6756e+05
Df Model:           9                F-statistic:        4.703e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.908            Scale:              1.4660e+06 
---------------------------------------------------------------------
          Coef.     Std.Err.     t      P>|t|     [0.025      0.975] 
---------------------------------------------------------------------
const    3928.6813    5.8287  674.0213  0.0000   3917.2569  3940.1057
x1       5140.0874   27.9240  184.0743  0.0000   5085.3558  5194.8189
x2        130.7076    7.0974   18.4163  0.0000    116.7966   144.6186
x3        551.4689  

Метрики из предыдущего пункта:

Test RMSE = 1240.1860; Test R^2 = 0.9050. Коэффициент детерминации R^2 = 0.905 высок, близок к единице, что говорит о высоком качестве построенной модели, то есть о значительном соответствии данным. RMSE = 1240.1860 не так велик относительно величины самих данных.

Метрики и выводы из таблицы:

- Судя по P-value, переменные x8 и x9 незначимы;
- Durbin-Watson = 2.004 => автокорреляция отсутствует;
- Prob (F-statistic) = 0 => модель адекватна, гипотезе о равенстве всех оцененых коэффициентов нулю не подтвердится;
- Omnibus = 9396.779, Prob(Omnibus) = 0.000, вероятность нормального распределения остатков нулевая;

In [97]:
print("Linear Regression with L1 regularization:\n", lasso_results.summary2())

Linear Regression with L1 regularization:
                   Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.907      
Dependent Variable: price            AIC:                735147.5726
Date:               2022-10-16 20:24 BIC:                735242.9699
No. Observations:   43152            Log-Likelihood:     -3.6756e+05
Df Model:           10               F-statistic:        4.233e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
Method:             elastic_net      Scale:              1.4660e+06 
R-squared:          0.908                                           
---------------------------------------------------------------------
          Coef.     Std.Err.     t      P>|t|     [0.025      0.975] 
---------------------------------------------------------------------
const    3928.6813    5.8287  674.0213  0.0000   3917.2569  3940.1057
x1       5140.0874   27.9240  184.0743  0.0000   5085.3558  5194.8189
x2   

Метрики из предыдущего пункта:
Test RMSE = 1250.8344; Test R^2 = 0.9034 Коэффициент детерминации R^2 высок, близок к единице, что говорит о высоком качестве построенной модели, то есть о значительном соответствии данным. RMSE не так велик относительно величины самих данных.

Метрики и выводы из таблицы:
- Судя по P-value, переменные x8 и x9 незначимы;
- Durbin-Watson = 2.004 => автокорреляция отсутствует;
- Prob (F-statistic) = 0 => модель адекватна, гипотезе о равенстве всех оцененых коэффициентов нулю не подтвердится;
- Omnibus = 9396.779, Prob(Omnibus) = 0.000, вероятность нормального распределения остатков нулевая;

In [98]:
from statsmodels.tools.tools import pinv_extended
pinv_wexog,_ = pinv_extended(ridge_regression.wexog)
normalized_cov_params = np.dot(pinv_wexog, np.transpose(pinv_wexog))
print("\nLinear Regression with L2 reqularization:\n", sm.regression.linear_model.OLSResults(ridge_regression,ridge_results.params,normalized_cov_params).summary2())


Linear Regression with L2 reqularization:
                   Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.905      
Dependent Variable: price            AIC:                736400.9569
Date:               2022-10-16 20:24 BIC:                736487.6818
No. Observations:   43152            Log-Likelihood:     -3.6819e+05
Df Model:           9                F-statistic:        4.554e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
R-squared:          0.905            Scale:              1.5093e+06 
---------------------------------------------------------------------
            Coef.    Std.Err.     t      P>|t|     [0.025     0.975] 
---------------------------------------------------------------------
const     3889.7835    5.9141  657.7108  0.0000  3878.1917  3901.3753
x1        4163.1002   28.3331  146.9339  0.0000  4107.5667  4218.6337
x2         133.7020    7.2014   18.5662  0.0000   119.5872   147.8169
x3 

Метрики из предыдущего пункта:

Test RMSE = 1240.1860; Test R^2 = 0.9050. Коэффициент детерминации R^2 = 0.905 высок, близок к единице, что говорит о высоком качестве построенной модели, то есть о значительном соответствии данным. RMSE = 1240.1860 не так велик относительно величины самих данных.

Метрики и выводы из таблицы:

- Судя по P-value, переменные x8 и x9 незначимы;
- Durbin-Watson = 2.002 => автокорреляция отсутствует;
- Prob (F-statistic) = 0 => модель адекватна, гипотезе о равенстве всех оцененых коэффициентов нулю не подтвердится;
- Omnibus = 9539.038, Prob(Omnibus) = 0.000, вероятность нормального распределения остатков нулевая;

In [99]:
print("\nLinear Regression with ElasticNet regularization:\n", elastic_results.summary2())


Linear Regression with ElasticNet regularization:
                   Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.907      
Dependent Variable: price            AIC:                735147.5726
Date:               2022-10-16 20:24 BIC:                735242.9699
No. Observations:   43152            Log-Likelihood:     -3.6756e+05
Df Model:           10               F-statistic:        4.233e+04  
Df Residuals:       43142            Prob (F-statistic): 0.00       
Method:             elastic_net      Scale:              1.4660e+06 
R-squared:          0.908                                           
---------------------------------------------------------------------
          Coef.     Std.Err.     t      P>|t|     [0.025      0.975] 
---------------------------------------------------------------------
const    3928.6813    5.8287  674.0213  0.0000   3917.2569  3940.1057
x1       5140.0874   27.9240  184.0743  0.0000   5085.3558  5194.8

Метрики из предыдущего пункта:

Test RMSE = 1240.1860; Test R^2 = 0.9050. Коэффициент детерминации R^2 = 0.905 высок, близок к единице, что говорит о высоком качестве построенной модели, то есть о значительном соответствии данным. RMSE = 1240.1860 не так велик относительно величины самих данных.

Метрики и выводы из таблицы:

- Судя по P-value, переменные x8 и x9 незначимы;
- Durbin-Watson = 2.004 => автокорреляция отсутствует;
- Prob (F-statistic) = 0 => модель адекватна, гипотезе о равенстве всех оцененых коэффициентов нулю не подтвердится;
- Omnibus = 9396.779, Prob(Omnibus) = 0.000, вероятность нормального распределения остатков нулевая;

Сравним полученные модели по оставшимся метрикам.

AIC BIC - Меньший AIC и BIC у первой модели, это позволяет только сказать, что в сравнении с остальными тремя она хуже переобучена, то есть лучше.

По R^2 и RMSE можно сказать, что модели схожи, разве что вторая показывает себя чуть хуже.

несмещённому R^2 в таблицах от statmodels, то чуть хуже по качеству себя показывает третья модель.

#### 4. [1 point] Implement one of the elimination algorithms that were described in the Seminar_4 (Elimination by P-value, Forward elimination, Backward elimination), make conclusions.

In [100]:
# Forward elimination by adjusted R squared.
variables = {}
for i in range(1, X_train_with_const.shape[1]):
  variables[i] = X_train_with_const[:,i]

# Constant from the beginning
base_X = X_train_with_const[:,0]

best_r_squared_adjusted = -1
best_variable_index = -1

while True:
  best_variable_index = -1
  for key in variables:
    temp_X = base_X.copy()
    temp_X = np.column_stack((temp_X, variables[key]))
    model = sm.OLS(y_train, temp_X).fit()

    #if(round(model.rsquared_adj,5) > best_r_squared_adjusted):
    if(model.rsquared_adj > best_r_squared_adjusted):
      best_r_squared_adjusted = model.rsquared_adj
      best_variable_index = key
  if best_variable_index == -1:
    break
  else:
    base_X = np.column_stack((base_X, variables[best_variable_index]))
    print(best_r_squared_adjusted)
    variables.pop(best_variable_index)

# base_X contains all variables which upgrade adjusted R squared metric.
# Every iteration code prints the best adjusted R squared.

0.8491996358008097
0.8857580227879949
0.9026002165823088
0.9047856615075786
0.9067972180905649
0.9073073202231309
0.9074804102461187
0.9074835724317534


Изначально в итеративном наборе храним только единицы для констант. Потом для каждой переменной проверяем: увеличится ли несмещённый коэффициент детерминации если добавить этот признак к модели. Если какой-то признак увеличил adjusted R^2 запоминаем его клюс в словаре, если он дал максимальный прирост окончательно добавляем его в набор и удаляем из словаря для перебора.

Программа вывела максимальные adjusted R^2 на каждой итерации, видно, что числа идут в порядке возрастания, как и положено. Заметим, что один признак не попал в итоговый набор, а последний признак набор увеличил метрику лишь на 0.0000035724317534, вполне возможно, что этот и пропущеный признаки и есть те самые два признака, которые оказались незначимы, как это было описано выше. Поэтому в коде оставляю закомментированную строку, которая при сравнении округляет число до 5 знаков после запятой, тогда последний признак не попадёт в лучший набор признаков.

#### 5. [1 point] Find the best (in terms of RMSE) $\alpha$ for Lasso regression using cross-validation with 4 folds. You must select values from range $[10^{-4}, 10^{3}]$.

In [101]:
# Lasso is a sklearn.linear_model
lasso_grid_search = GridSearchCV(Lasso(), {'alpha':np.logspace(-4, 3, num = 50, base = 10)}, scoring='neg_root_mean_squared_error', cv=4)
lasso_grid_search.fit(X_train_with_const, y_train)
print(lasso_grid_search.best_params_)

{'alpha': 1.9306977288832496}


## Gradient descent

#### 6. [3.5 points] Implement a Ridge regression model for the MSE loss function, trained by gradient descent.

All calculations must be vectorized, and python loops can only be used for gradient descent iterations. As a stop criterion, you must use (simultaneously):

* checking for the Absolute-value norm of the weight difference on two adjacent iterations (for example, less than some small number of the order of $10^{-6}$, set by the `tolerance` parameter);
* reaching the maximum number of iterations (for example, 10000, set by the `max_iter` parameter).

You need to implement:

* Full gradient descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

* Stochastic Gradient Descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} q_{i_{k}}(w_{k}).
$$

$\nabla_{w} q_{i_{k}}(w_{k}) \, $ is the estimate of the gradient over the batch of objects selected randomly.

* Momentum method:

$$
h_0 = 0, \\
h_{k + 1} = \alpha h_{k} + \eta_k \nabla_{w} Q(w_{k}), \\
w_{k + 1} = w_{k} - h_{k + 1}.
$$

* Adagrad method:

$$
G_0 = 0, \\
G_{k + 1} = G_{k} + (\nabla_{w} Q(w_{k+1}))^2, \\
w_{k + 1} = w_{k} - \eta * \frac{\nabla_{w} Q(w_{k+1})}{\sqrt{G_{k+1} + \epsilon}}.
$$



To make sure that the optimization process really converges, we will use the `loss_history` class attribute. After calling the `fit` method, it should contain the values of the loss function for all iterations, starting from the first one (before the first step on the anti-gradient).

You need to initialize the weights with a random vector from normal distribution. The following is a template class that needs to contain the code implementing all variations of the models.

In [105]:
from sklearn.base import BaseEstimator

class LinReg(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='Momentum', 
                 tolerance=1e-4, max_iter=1000, w0=None, eta=1e-2, alpha=1e-3):
        """
        gd_type: str
            'GradientDescent', 'StochasticDescent', 'Momentum', 'Adagrad'
        delta: float
            proportion of object in a batch (for stochastic GD)
        tolerance: float
            for stopping gradient descent
        max_iter: int
            maximum number of steps in gradient descent
        w0: np.array of shape (d)
            init weights
        eta: float
            learning rate
        alpha: float
            momentum coefficient
        reg_cf: float
            regularization coefficient
        epsilon: float
            numerical stability
        """
        
        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.loss_history = None # list of loss function values at each training iteration
    
    def fit(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: self
        """
        self.loss_history = []
        n, k = X.shape
        if self.w0 is None:
          self.w = np.random.normal(0, 1, k+1)
        else:
          self.w = self.w0
        return self
    
    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        return np.dot(X, self.w)
    
    def calc_gradient(self, X, y):
      match(self.gd_type):
          case 'GradientDescent':
              print('Hello to you too!')
          case 'StochasticDescent':
              print('Hello to you too!')
          case 'Momentum':
              print('Hello to you too!')
          case 'Adagrad':
              print('Hello to you too!')

        """
        X: np.array of shape (l, d) (l can be equal to 1 if stochastic)
        y: np.array of shape (l)
        ---
        output: np.array of shape (d)
        """

    def calc_loss(self, X, y):
        error = y-self.predict(X)
        return 1.0/(y.size)*np.dot(error.T, error)

IndentationError: ignored

#### 7. [1 points] Train and validate "hand-written" models on the same data, and compare the quality with the Sklearn or StatsModels methods. Investigate the effect of the `max_iter` and `alpha` parameters on the optimization process. Is it consistent with your expectations?

In [ ]:
# your code here 
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

#### 8. [1 points] Plot graphs (on the same picture) of the dependence of the loss function value on the iteration number for Full GD, SGD, Momentum and Adagrad. Draw conclusions about the rate of convergence of various modifications of gradient descent.

Don't forget about what *beautiful* graphics should look like!

In [ ]:
# your code here 
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ